In [ ]:
"""
================================================================================
SCRIPT: Log_transform_NonSkewed_LightGBM.py
ESP32 Heater ML Pipeline - NON-SKEWED (LIGHTGBM UPGRADE)
================================================================================

PURPOSE:
This script upgrades the algorithm to LightGBM (LGBMRegressor) while retaining the
final, corrected feature set (thermal-only) and the non-skewed log transformation.
The goal is to increase R-squared and lower MAE by using a more powerful algorithm
to learn the subtle thermal dynamics.

TRANSFORMATION FORMULA: y = log(1 + PWM_Target)
ALGORITHM: LightGBM (LGBMRegressor)

================================================================================
"""

import pandas as pd
import numpy as np
import glob
import os
import lightgbm as lgb # --- NEW ALGORITHM IMPORT ---
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import joblib

# --- Configuration Settings (ADJUST THESE) ---
CSV_DIRECTORY_PATH = '.'
PREDICTION_WINDOW_STEPS = 10
LAG_STEPS_TEMP = [1, 5, 10]
LAG_STEPS_PWM = [5, 10]
# ---------------------------------------------

# Mapping the raw column names from the ESP32 log files
MAP_FEATURES = {
    'thrmstr6': 'T_Heater',
    'thrmstrout': 'T_Outside',
    'bme_temp': 'T_Room_BME',
    'pwm': 'PWM_Applied',
    'activity_sim': 'T_Target_Demand'
}

# ==========================================================
# Step 1 & 2: Load Data and Basic Cleaning
# ==========================================================
csv_file_pattern = os.path.join(CSV_DIRECTORY_PATH, '*.csv')
all_csv_files = glob.glob(csv_file_pattern)

file_names = [f for f in all_csv_files
              if os.path.basename(f).startswith('Dynamic_') or
                 os.path.basename(f).startswith('Stabilized_')]

if not file_names:
    raise ValueError(f"No CSV files found matching the criteria in: {CSV_DIRECTORY_PATH}")

df_list = []
for f in file_names:
    df = pd.read_csv(f)
    # Tagging each row with its source file for the Run-Aware Split (Leakage Fix)
    df['source_file'] = os.path.basename(f)
    df_list.append(df)

combined_df = pd.concat(df_list, ignore_index=True)
combined_df.columns = [c.strip().lower() for c in combined_df.columns]
combined_df.rename(columns=MAP_FEATURES, inplace=True)
print(f"Combined DataFrame shape: {combined_df.shape}")

# Ensure T_Target_Demand exists (for older logs without simulated occupancy)
if 'T_Target_Demand' not in combined_df.columns:
    combined_df['T_Target_Demand'] = 25.0

# ==========================================================
# Step 3: Feature Engineering (Lagged & Delta Features)
# ==========================================================
def feature_engineer_time_series(df):
    """Generates the target (shifted) and lagged features."""

    # CRITICAL: Target is the PWM 10 steps (10 minutes) into the future
    df['PWM_Target'] = df['PWM_Applied'].shift(-PREDICTION_WINDOW_STEPS)

    # Generate Lagged Temperature Features (T_Heater and T_Room_BME)
    for col in ['T_Heater', 'T_Room_BME']:
        for lag in LAG_STEPS_TEMP:
            df[f'{col}_lag_{lag}min'] = df[col].shift(lag)

    # Generate Lagged PWM Features (THESE ARE STILL CALCULATED BUT NOT USED LATER)
    for lag in LAG_STEPS_PWM:
        df[f'PWM_Applied_lag_{lag}min'] = df['PWM_Applied'].shift(lag)

    # Generate Differential Feature (Rate of Change)
    df['T_Heater_Delta_5min'] = df['T_Heater'] - df['T_Heater'].shift(5)

    # Drop rows where target or the longest lagged features are NaN (created by the shifts)
    df.dropna(subset=['PWM_Target', 'T_Heater_lag_10min'], inplace=True)
    return df

combined_df_fe = feature_engineer_time_series(combined_df.copy())
print(f"DataFrame shape after feature engineering: {combined_df_fe.shape}")

# ==========================================================
# Step 4: Stage B - Predict Non-Skewed Log-Transformed PWM Control
# ==========================================================

# 1. Define Final Optimized Feature List (CLEAN - ONLY THERMAL FEATURES)
feature_cols_stage_b_optimized = [
    'T_Heater', 'T_Outside', 'T_Room_BME', 'T_Target_Demand', 'T_Heater_Delta_5min',
    'T_Heater_lag_1min', 'T_Heater_lag_5min', 'T_Heater_lag_10min',
    'T_Room_BME_lag_1min', 'T_Room_BME_lag_5min', 'T_Room_BME_lag_10min',
]

X = combined_df_fe[feature_cols_stage_b_optimized]

# --- ORIGINAL NON-SKEWED LOG TRANSFORMATION ---
# Original Target: y = log(1 + PWM_Target)
y = np.log1p(combined_df_fe['PWM_Target'])
print(f"Target variable created using Original Non-Skewed Log Transformation.")


# 2. Train/Test Split by Runs (Temporal Leakage Fix)
unique_runs = combined_df_fe['source_file'].unique()
train_runs, test_runs = train_test_split(unique_runs, test_size=0.265, random_state=42)

train_mask_b = combined_df_fe['source_file'].isin(train_runs)
test_mask_b = combined_df_fe['source_file'].isin(test_runs)

X_train_b, X_test_b = X[train_mask_b].copy(), X[test_mask_b].copy()
y_train_b, y_test_b = y[train_mask_b].copy(), y[test_mask_b].copy()

# 3. --- CRITICAL LEAKAGE FIX: Impute missing values using only the training set mean ---
print("\nApplying Imputation Leakage Fix...")
imputation_cols = ['T_Heater', 'T_Outside', 'T_Room_BME']

for col in imputation_cols:
    train_mean = X_train_b[col].mean()
    X_train_b[col] = X_train_b[col].fillna(train_mean)
    X_test_b[col] = X_test_b[col].fillna(train_mean)

print(f"Stage B Training shape: {X_train_b.shape}, Testing shape: {X_test_b.shape}")

# 4. Train LightGBM REGRESSOR
print("\nTraining Stage B LightGBM REGRESSOR (Forced Thermal Learning)...")
lgbm_regressor = lgb.LGBMRegressor(
    objective='regression_l1', # Use MAE objective
    n_estimators=1000,
    learning_rate=0.05,
    n_jobs=-1,
    random_state=42,
    metric='mae'
)
lgbm_regressor.fit(
    X_train_b, y_train_b,
    eval_set=[(X_test_b, y_test_b)],
    eval_metric='mae',
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

# 5. Evaluate Stage B
y_pred_b_log = lgbm_regressor.predict(X_test_b)

# --- ORIGINAL INVERSE TRANSFORMATION ---
# Formula: PWM = e^y - 1
y_pred_b_linear = np.expm1(y_pred_b_log)
y_test_b_linear = np.expm1(y_test_b)


mae_b = mean_absolute_error(y_test_b_linear, y_pred_b_linear)
rmse_b = np.sqrt(mean_squared_error(y_test_b_linear, y_pred_b_linear))
r2_score_linear = r2_score(y_test_b_linear, y_pred_b_linear)

print("\n--- Stage B Test Performance (LightGBM - Non-Skewed Log) ---")
print(f"Mean Absolute Error (MAE): {mae_b:.4f}")
print(f"Root Mean Square Error (RMSE): {rmse_b:.4f}")
print(f"R-squared (R2, Linear): {r2_score_linear:.4f}")

# 6. Feature Importance and Model Saving
importances = lgbm_regressor.feature_importances_
importance_df = pd.DataFrame({'Feature': feature_cols_stage_b_optimized, 'Importance': importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("\nStage B Feature Importances (Top 5):")
# Note: LightGBM returns raw counts, so normalize to percentage for comparison
total_importance = importance_df['Importance'].sum()
importance_df['Importance (%)'] = (importance_df['Importance'] / total_importance) * 100
print(importance_df.head(5).to_string(index=False))

# Save the final model
model_filename = 'lgbm_proactive_heater_regressor_NON_SKEWED_THERMAL.joblib'
joblib.dump(lgbm_regressor, model_filename)
print(f"\nFinal LightGBM Regressor Model saved as: {model_filename}")

print("\n==========================================================")
print("Pipeline Complete: LightGBM Applied to Proactive Feature Set.")
print("==========================================================")










Combined DataFrame shape: (287780, 17)
DataFrame shape after feature engineering: (287760, 28)
Target variable created using Original Non-Skewed Log Transformation.

Applying Imputation Leakage Fix...
Stage B Training shape: (201638, 11), Testing shape: (86122, 11)

Training Stage B LightGBM REGRESSOR (Forced Thermal Learning)...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011717 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2406
[LightGBM] [Info] Number of data points in the train set: 201638, number of used features: 10
[LightGBM] [Info] Start training from score 3.433987

--- Stage B Test Performance (LightGBM - Non-Skewed Log) ---
Mean Absolute Error (MAE): 97.9212
Root Mean Square Error (RMSE): 201.1743
R-squared (R2, Linear): 0.4510

Stage B Feature Importances (Top 5):
             Feature  Importance  Importance (%)
      